<a href="https://colab.research.google.com/github/OUCTheoryGroup/colab_demo/blob/master/05_05_GoogLeNet_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GoogLeNet 应用于 CIFAR10 分类

2014年，GoogLeNet和VGG是当年ImageNet挑战赛的双雄，GoogLeNet获得了第一名、VGG获得了第二名，这两类模型结构的共同特点是层次更深了。GoogLeNet则做了更加大胆的网络结构尝试，虽然深度只有22层，但大小却比AlexNet和VGG小很多，GoogleNet参数为500万个，AlexNet参数个数是GoogleNet的12倍，VGGNet参数又是AlexNet的3倍，因此在内存或计算资源有限时，GoogleNet是比较好的选择；从模型结果来看，GoogLeNet的性能却更加优越。

**小知识：**GoogLeNet是谷歌（Google）研究出来的深度网络结构，为什么不叫“GoogleNet”，而叫“GoogLeNet”，据说是为了向“LeNet”致敬，因此取名为“GoogLeNet”

有没有一种方法既能保持网络结构的稀疏性，又能利用密集矩阵的高计算性能。大量的文献表明可以将稀疏矩阵聚类为较为密集的子矩阵来提高计算性能，就如人类的大脑是可以看做是神经元的重复堆积，因此，GoogLeNet团队提出了Inception网络结构，就是构造一种“基础神经元”结构，来搭建一个稀疏性、高计算性能的网络结构。

**小知识：**Inception 的名字来自于电影《盗梦空间》，电影英文名就是Inception，影片有一句台词：“We need to go deeper.” 即在神经网络中，需要“go deeper”。

Inception历经了V1、V2、V3、V4等多个版本的发展，不断趋于完善。今天我们编程，选择的是 InceptionV3，根据Google官方，可以得到 InceptionV3 结构如下：

![InceptionV3](http://q6dz4bbgt.bkt.clouddn.com/20200229182425.jpg)

下面，我们编码实现这个网络。

## 1. 定义 dataloader 

这部分代码和之前 VGG 那个教程是完全一样的。

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 使用GPU训练，可以在菜单 "代码执行工具" -> "更改运行时类型" 里进行设置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform_train)
testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 2. 定义 InceptionV3 网络

首先定义一个基本卷积类，这个在后面会大量使用。可以看到，这个类把 卷积、BatchNormalization、ReLU 激活，打包在了一起，非常容易理解。

In [0]:
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

### 2.1 Inception A 模块

从 InceptionV3 总体架构图中可以看出，第一个部分为 Inception A 模块，该模块的示意如下所示：

![Inception A 模块示意图](http://q6dz4bbgt.bkt.clouddn.com/20200229204301.jpg)

该结构将CNN中常用的卷积（1x1，3x3，5x5）堆叠在一起（卷积后的尺寸相同，将通道相加），一方面增加了网络的宽度，另一方面也增加了网络对尺度的适应性。

**请大家思考一个问题：1x1的卷积核有什么用呢？**

1x1卷积的主要目的是为了减少维度，具体大家可以参考知乎一个文章《[1X1卷积核的作用](https://zhuanlan.zhihu.com/p/37910136)》

该模块的代码如下：

In [0]:
class InceptionA(nn.Module):

    def __init__(self, in_channels, pool_features):
        super(InceptionA, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(48, 64, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(96, 96, kernel_size=3, padding=1)

        self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

下面写一段非常非常简的代码来分析一下 Inception A 模块：

In [6]:
# 初始化
net = InceptionA(3, 32)
# 随机生成一个feature map 大小为  3*32*32 （feature map 都是四维的，第一个维度是样本数量，人们在论文介绍中往往忽略这个样本数量）
x = torch.randn(1, 3, 32, 32)
y = net(x)
print('Feature map size after Inception A: ', y.size())

Feature map size after Inception A:  torch.Size([1, 256, 32, 32])


可以看出，现在的 feature map 尺寸是： 256 x 32 x 32 。

我们分析下这个256是怎么来的，代码中可以看出来，四个分支得到 feature map 数量分别是：
 - branch1x1 ==> 64
 - branch5x5 ==> 64
 - branch3x3dbl ==> 96
 - branch_pool ==> pool_features (这个是输入参数)
 
 得到的特征图数量一共是：

64 + 64 + 96 + pool_features = 224 + pool_features 

可以看出， pool_featurs 主要起到一个连接前后两层的目的。

### 2.2 Inception B 模块

从 InceptionV3 总体架构图中可以看出，第二个部分为 Inception B 模块，该模块的示意如下所示：

![InceptionB](http://img.huaiwen.me/20190119231001.jpg)

该模块的代码如下：

In [0]:
class InceptionB(nn.Module):

    def __init__(self, in_channels):
        super(InceptionB, self).__init__()
        self.branch3x3 = BasicConv2d(in_channels, 384, kernel_size=3, stride=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(96, 96, kernel_size=3, stride=2)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

和上面一样，下面继续写一段非常非常简的代码来分析一下 Inception B 模块：

In [0]:
net = InceptionB(128)
x = torch.randn(1, 128, 32, 32)
y = net(x)
print('Feature map size after Inception B: ', y.size())

Feature map size after Inception B:  torch.Size([1, 608, 15, 15])


需要注意的一点是，Inception B 模块需要一个输入参数是 in_channels，也就是说，这个参数必须和输入数据的通道数一致，不然运行的过程中就会报错。

从总体架构图中看，Inception B 这个模块使用频率并不高，只出现过一次。它的作用是什么呢？

它的作用是让图像的尺寸变小。从上面的演示可以看出，feature map 尺寸由 32 变成了 15，相当于池化加了一个Inception A 模块。但这样计算量会比较大，为了同时保持特征表示且降低计算量，谷歌就设计了 Inception B 模块，使用两个并行化的模块来降低计算量（卷积、池化并行执行，再进行合并）

通过代码可以看出，三个分支得到的 feature map 数量依次是：
- branch3x3 ===> 384
- branch3x3dbl ==> 96
- branch_pool ==> in_channels

所以，输出的 feature map 数量是 384 + 96 + in_channels = 480 + in_channels，尺寸为原来的 1/2。

### 2.3 Inception C 模块

从 InceptionV3 总体架构图中可以看出，第三个部分为 Inception C 模块，该模块的示意如下所示：

![InceptionC](http://img.huaiwen.me/20190119231031.jpg)

任意nxn的卷积都可以通过1xn卷积后接nx1卷积来替代（可以降低计算量），具体原因可以百度一下。GoogLeNet团队发现在网络的前期使用这种分解效果并不好，在中度大小的特征图（feature map）上使用效果才会更好（特征图大小建议在12到20之间）。

从图中可以看出，里面有大量使用 7x1 和 1x7 来取代 7x7 的卷积核。

该模块的代码如下：

In [0]:
class InceptionC(nn.Module):

    def __init__(self, in_channels, channels_7x7):
        super(InceptionC, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 192, kernel_size=1)

        c7 = channels_7x7
        self.branch7x7_1 = BasicConv2d(in_channels, c7, kernel_size=1)
        self.branch7x7_2 = BasicConv2d(c7, c7, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7_3 = BasicConv2d(c7, 192, kernel_size=(7, 1), padding=(3, 0))

        self.branch7x7dbl_1 = BasicConv2d(in_channels, c7, kernel_size=1)
        self.branch7x7dbl_2 = BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7dbl_3 = BasicConv2d(c7, c7, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7dbl_4 = BasicConv2d(c7, c7, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7dbl_5 = BasicConv2d(c7, 192, kernel_size=(1, 7), padding=(0, 3))

        self.branch_pool = BasicConv2d(in_channels, 192, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch7x7 = self.branch7x7_1(x)
        branch7x7 = self.branch7x7_2(branch7x7)
        branch7x7 = self.branch7x7_3(branch7x7)

        branch7x7dbl = self.branch7x7dbl_1(x)
        branch7x7dbl = self.branch7x7dbl_2(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_3(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_4(branch7x7dbl)
        branch7x7dbl = self.branch7x7dbl_5(branch7x7dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch7x7, branch7x7dbl, branch_pool]
        return torch.cat(outputs, 1)

和上面一样，下面继续写一段非常非常简的代码来分析一下 Inception C 模块：

In [0]:
net = InceptionC(128, channels_7x7=128)
x = torch.randn(1, 128, 32, 32)
y = net(x)
print('Feature map size after Inception C: ', y.size())

Feature map size after Inception C:  torch.Size([1, 768, 32, 32])


可以看到，图像的大小不会发生变化，四个分支得到的 feature map 数量都是192，因此，经过这个模块，feature map 的数量永远是 768。

### 2.4 Inception D 模块

这个模块的示意如下所示：

![InceptionD](http://img.huaiwen.me/20190119231104.jpg)

下面是具体代码：



In [0]:
class InceptionD(nn.Module):

    def __init__(self, in_channels):
        super(InceptionD, self).__init__()
        self.branch3x3_1 = BasicConv2d(in_channels, 192, kernel_size=1)
        self.branch3x3_2 = BasicConv2d(192, 320, kernel_size=3, stride=2)

        self.branch7x7x3_1 = BasicConv2d(in_channels, 192, kernel_size=1)
        self.branch7x7x3_2 = BasicConv2d(192, 192, kernel_size=(1, 7), padding=(0, 3))
        self.branch7x7x3_3 = BasicConv2d(192, 192, kernel_size=(7, 1), padding=(3, 0))
        self.branch7x7x3_4 = BasicConv2d(192, 192, kernel_size=3, stride=2)

    def forward(self, x):
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)

        branch7x7x3 = self.branch7x7x3_1(x)
        branch7x7x3 = self.branch7x7x3_2(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_3(branch7x7x3)
        branch7x7x3 = self.branch7x7x3_4(branch7x7x3)

        branch_pool = F.max_pool2d(x, kernel_size=3, stride=2)
        outputs = [branch3x3, branch7x7x3, branch_pool]
        return torch.cat(outputs, 1)

我们继续用一段代码来分析这个模块：

In [10]:
net = InceptionD(128)
x = torch.randn(1, 128, 32, 32)
y = net(x)
print('Feature map size after Inception D: ', y.size())

Feature map size after Inception D:  torch.Size([1, 640, 15, 15])


通过代码可以看出，三个分支得到的 feature map 数量依次是：
- branch3x3 ===> 320
- branch7x7x3 ==> 192
- branch_pool ==> in_channels

因此，总特征图的数量为：512+in_channels，尺寸为原来的一半

### 2.5 Inception E 模块

这个模块的示意如下所示：

![InceptionE](http://img.huaiwen.me/20190119231132.jpg)

下面是具体代码：


In [0]:
class InceptionE(nn.Module):

    def __init__(self, in_channels):
        super(InceptionE, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 320, kernel_size=1)

        self.branch3x3_1 = BasicConv2d(in_channels, 384, kernel_size=1)
        self.branch3x3_2a = BasicConv2d(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3_2b = BasicConv2d(384, 384, kernel_size=(3, 1), padding=(1, 0))

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 448, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(448, 384, kernel_size=3, padding=1)
        self.branch3x3dbl_3a = BasicConv2d(384, 384, kernel_size=(1, 3), padding=(0, 1))
        self.branch3x3dbl_3b = BasicConv2d(384, 384, kernel_size=(3, 1), padding=(1, 0))

        self.branch_pool = BasicConv2d(in_channels, 192, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch3x3 = self.branch3x3_1(x)
        branch3x3 = [
            self.branch3x3_2a(branch3x3),
            self.branch3x3_2b(branch3x3),
        ]
        branch3x3 = torch.cat(branch3x3, 1)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = [
            self.branch3x3dbl_3a(branch3x3dbl),
            self.branch3x3dbl_3b(branch3x3dbl),
        ]
        branch3x3dbl = torch.cat(branch3x3dbl, 1)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

继续用代码分析模块的输出：

In [12]:
net = InceptionE(128)
x = torch.randn(1, 128, 32, 32)
y = net(x)
print('Feature map size after Inception E: ', y.size())

Feature map size after Inception E:  torch.Size([1, 2048, 32, 32])


通过代码可以看出，四个分支得到的 feature map 数量依次是：
- branch1x1 ===> 320
- branch3x3 ==> 768
- branch3x3dbl ==> 768
- branch_pool ==> 192

因此，总的 feature map 数量为 2048，图像尺寸不变

本来还有一个 Inception Auxiliary Classifiers，这里不过多介绍了。

## 3. 整体架构

网络的整体架构如图：

![InceptionV3](http://q6dz4bbgt.bkt.clouddn.com/20200229182425.jpg)

原版的网络实是在太大了，难以放到GPU上，受 colab 的 GPU资源限制，这里我对网重复的模块进行了简化 ~~~ 代码如下：

In [0]:
class Inception3(nn.Module):

    def __init__(self, num_classes=1000):
        super(Inception3, self).__init__()
        self.Conv2d_1a_3x3 = BasicConv2d(3, 32, kernel_size=3, stride=2)
        self.Conv2d_2a_3x3 = BasicConv2d(32, 32, kernel_size=3)
        self.Mixed_A = InceptionA(32, pool_features=32)
        self.Mixed_B = InceptionB(256)
        self.Mixed_C = InceptionC(736, channels_7x7=128)
        self.Mixed_D = InceptionD(768)
        self.Mixed_E = InceptionE(1280)
        self.fc = nn.Linear(2048, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                stddev = m.stddev if hasattr(m, 'stddev') else 0.1
                X = stats.truncnorm(-2, 2, scale=stddev)
                values = torch.Tensor(X.rvs(m.weight.numel()))
                values = values.view(m.weight.size())
                m.weight.data.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.Conv2d_1a_3x3(x)
        x = self.Conv2d_2a_3x3(x)
        x = self.Mixed_A(x)
        x = self.Mixed_B(x)    
        x = self.Mixed_C(x)  
        x = self.Mixed_D(x)   
        x = self.Mixed_E(x)
        # 2 x 2 x 2048
        x = F.avg_pool2d(x, kernel_size=2)
        # 1 x 1 x 2048
        x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048
        x = self.fc(x)
        return x

网络初始化，具体代码如下：

In [0]:
# 网络放到GPU上
net = Inception3( num_classes=10 ).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

## 4. 网络训练

训练的代码和之前是完全一样的：

In [38]:
for epoch in range(10):  # 重复多轮训练
    for i, (inputs, labels) in enumerate(trainloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化 
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # 输出统计信息
        if i % 100 == 0:   
            print('Epoch: %d Minibatch: %5d loss: %.3f' %(epoch + 1, i + 1, loss.item()))

print('Finished Training')

Epoch: 1 Minibatch:     1 loss: 0.361
Epoch: 1 Minibatch:   101 loss: 0.366
Epoch: 1 Minibatch:   201 loss: 0.375
Epoch: 1 Minibatch:   301 loss: 0.435
Epoch: 2 Minibatch:     1 loss: 0.481
Epoch: 2 Minibatch:   101 loss: 0.380
Epoch: 2 Minibatch:   201 loss: 0.443
Epoch: 2 Minibatch:   301 loss: 0.485
Epoch: 3 Minibatch:     1 loss: 0.364
Epoch: 3 Minibatch:   101 loss: 0.430
Epoch: 3 Minibatch:   201 loss: 0.426
Epoch: 3 Minibatch:   301 loss: 0.314
Epoch: 4 Minibatch:     1 loss: 0.279
Epoch: 4 Minibatch:   101 loss: 0.248
Epoch: 4 Minibatch:   201 loss: 0.279
Epoch: 4 Minibatch:   301 loss: 0.392
Epoch: 5 Minibatch:     1 loss: 0.423
Epoch: 5 Minibatch:   101 loss: 0.334
Epoch: 5 Minibatch:   201 loss: 0.437
Epoch: 5 Minibatch:   301 loss: 0.462
Epoch: 6 Minibatch:     1 loss: 0.308
Epoch: 6 Minibatch:   101 loss: 0.347
Epoch: 6 Minibatch:   201 loss: 0.368
Epoch: 6 Minibatch:   301 loss: 0.299
Epoch: 7 Minibatch:     1 loss: 0.291
Epoch: 7 Minibatch:   101 loss: 0.319
Epoch: 7 Min

## 5. 测试验证准确率

测试的代码和之前是完全一样的：

In [39]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 85.40 %


准确率又提高了一些，目前是因为 EPOCH 跑的较少，进一步多跑到150-200轮，准确率可以非常高。

大家可能好奇，Inception明明有更好的版本，这里为什么以 V3 为示例呢？

因为 V4 的主要改进是采用了 ResNet 的结构 ~~~ 

相信你也猜到了，下一节课的内容就是围绕 ResNet 展开。